In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import sys
import tensorflow as tf
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from datetime import timedelta
from tqdm import tqdm


In [20]:
df = pd.read_csv('C:\Jupyter_Project\HY.csv')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-02-04,6930.0,6980.0,6890.0,6930.0,5837.953613,19034.0
1,2016-02-05,6990.0,7190.0,6980.0,7040.0,5930.619629,23731.0
2,2016-02-11,6880.0,6880.0,6720.0,6800.0,5728.439941,18254.0
3,2016-02-12,6740.0,6740.0,6450.0,6600.0,5559.956055,43774.0
4,2016-02-15,6670.0,6800.0,6630.0,6800.0,5728.439941,26119.0


In [21]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scale_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
df_scaled = scaler.fit_transform(df[scale_cols])

df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = scale_cols

print(df_scaled)

          Open      High       Low     Close    Volume
0     0.411862  0.392617  0.438811  0.425210  0.006805
1     0.421746  0.427852  0.454545  0.443697  0.008485
2     0.403624  0.375839  0.409091  0.403361  0.006526
3     0.380560  0.352349  0.361888  0.369748  0.015650
4     0.369028  0.362416  0.393357  0.403361  0.009338
5     0.397035  0.369128  0.417832  0.406723  0.006459
6     0.392092  0.367450  0.412587  0.401681  0.004338
7     0.401977  0.379195  0.426573  0.416807  0.003704
8     0.406919  0.379195  0.414336  0.416807  0.004628
9     0.397035  0.379195  0.430070  0.420168  0.003179
10    0.421746  0.395973  0.431818  0.420168  0.007557
11    0.403624  0.384228  0.435315  0.423529  0.008429
12    0.408567  0.402685  0.442308  0.433613  0.004699
13    0.431631  0.412752  0.456294  0.448739  0.004621
14    0.434926  0.427852  0.470280  0.462185  0.004915
15    0.449753  0.432886  0.480769  0.467227  0.005654
16    0.456343  0.431208  0.484266  0.470588  0.002737
17    0.45

In [22]:
window_size = 10
TEST_SIZE = 26

In [23]:
train = df_scaled[:-TEST_SIZE]
test = df_scaled[-TEST_SIZE:]

In [24]:
# 정해진 window_size에 기반하여 10일 기간의 데이터 셋을 묶어준다

def make_dataset(data, label, window_size=10):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [25]:
from sklearn.model_selection import train_test_split

feature_cols = ['Open', 'High', 'Low', 'Volume']
label_cols = ['Close']

train_feature = train[feature_cols]
train_label = train[label_cols]

# train dataset
train_feature, train_label = make_dataset(train_feature, train_label, 10)

# train, validation set 생성
x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)

x_train.shape, x_valid.shape
# ((960, 20, 4), (240, 20, 4))

# test dataset (실제 예측 해볼 데이터)
test_feature = test[feature_cols]
test_label = test[label_cols]

test_feature.shape, test_label.shape
# ((26-10, 10, 4), (26-10, 1))

((26, 4), (26, 1))

In [26]:
test_feature, test_label = make_dataset(test_feature, test_label, 10)

test_feature.shape, test_label.shape

((16, 10, 4), (16, 1))

In [27]:
x_train.shape

(952, 10, 4)

In [28]:
x_valid.shape

(238, 10, 4)

In [30]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM

model = Sequential()
model.add(LSTM(16, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               return_sequences=False)
          )
model.add(Dense(1))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [31]:
model.compile(loss='mean_squared_error', optimizer='adam')
early_stop = EarlyStopping(monitor='val_loss', patience=10)
filename = os.path.join('tmp', 'ckeckpointer.ckpt')
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

history = model.fit(x_train, y_train, 
                    epochs=200, 
                    batch_size=1,
                    validation_data=(x_valid, y_valid), 
                    callbacks=[early_stop, checkpoint])


Train on 952 samples, validate on 238 samples
Epoch 1/200
952/952 [==============================] - 4s 4ms/step - loss: nan - val_loss: nan

Epoch 00001: val_loss did not improve from inf
Epoch 2/200
952/952 [==============================] - 3s 3ms/step - loss: nan - val_loss: nan

Epoch 00002: val_loss did not improve from inf
Epoch 3/200
952/952 [==============================] - 3s 3ms/step - loss: nan - val_loss: nan

Epoch 00003: val_loss did not improve from inf
Epoch 4/200
952/952 [==============================] - 3s 4ms/step - loss: nan - val_loss: nan

Epoch 00004: val_loss did not improve from inf
Epoch 5/200
952/952 [==============================] - 3s 4ms/step - loss: nan - val_loss: nan

Epoch 00005: val_loss did not improve from inf
Epoch 6/200
952/952 [==============================] - 3s 4ms/step - loss: nan - val_loss: nan

Epoch 00006: val_loss did not improve from inf
Epoch 7/200
952/952 [==============================] - 3s 3ms/step - loss: nan - val_loss: nan


In [32]:
model.load_weights(filename)
pred  = model.predict(test_feature)

pred.shape

OSError: Unable to open file (unable to open file: name = 'tmp\ckeckpointer.ckpt', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)